<a href="https://colab.research.google.com/github/GharmMDS/Sms_text_classification/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

In [ ]:
VOCAB_SIZE = 10000

vectorize_layer = layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='tf_idf',
    standardize="lower_and_strip_punctuation"
)

vectorize_layer.adapt(train_df['message'])

y_train = train_df['label'].apply(lambda x: 1 if x == 'spam' else 0)
y_test = test_df['label'].apply(lambda x: 1 if x == 'spam' else 0)

model = keras.Sequential([
    keras.Input(shape=(1,), dtype=tf.string),
    vectorize_layer,
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

train_ds = tf.data.Dataset.from_tensor_slices((train_df['message'], y_train)).batch(32).cache().prefetch(tf.data.AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((test_df['message'], y_test)).batch(32).cache().prefetch(tf.data.AUTOTUNE)

history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds,
    verbose=0
)

print("\nModel training complete.")

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    prediction_proba = model.predict(tf.constant([pred_text]))[0][0]
    label = ""
    if prediction_proba > 0.5:
        label = "spam"
    else:
        label = "ham"
    return [prediction_proba, label]

pred_text_example = "how are you doing today?"
prediction_result = predict_message(pred_text_example)
print(prediction_result)

pred_text_spam_example = "URGENT! Your Mobile number has been awarded a £2000 prize. Call 0906XXXXXXX"
prediction_spam_result = predict_message(pred_text_spam_example)
print(prediction_spam_result)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
